In [27]:
import os
import pickle as pkl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

In [28]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option("max_colwidth", None)
pd.set_option("max_seq_items", None)

In [29]:
MARCH_PARENT_DIR = "./cicddos_data/CSV-03-11/03-11"
JAN_PARENT_DIR = "./cicddos_data/CSV-01-12/01-12"

OUT_DIR = "./cicddos_data/out"

BIN_TRAIN_DATA_PATH = "./cicddos_data/out/bin-train.csv"
BIN_TEST_DATA_PATH = "./cicddos_data/out/bin-test.csv"
TRAIN_DATA_PATH = "./cicddos_data/out/train.csv"
TEST_DATA_PATH = "./cicddos_data/out/test.csv"

In [30]:
train_df = pd.read_csv(TRAIN_DATA_PATH, low_memory=False)
test_df = pd.read_csv(TEST_DATA_PATH, low_memory=False)

In [15]:
bin_train_df = pd.read_csv(BIN_TRAIN_DATA_PATH, low_memory=False)

In [17]:
bin_test_df = pd.read_csv(BIN_TEST_DATA_PATH, low_memory=False)

In [31]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [22]:
train_df['Label'].value_counts()

DrDoS_SSDP       199955
Syn              199948
DrDoS_SNMP       199944
DrDoS_NetBIOS    199904
DrDoS_MSSQL      199897
DrDoS_DNS        199868
DrDoS_LDAP       199855
DrDoS_UDP        199822
TFTP             199730
UDP-lag          197778
DrDoS_NTP        197597
BENIGN             5471
WebDDoS             231
Name: Label, dtype: int64

In [16]:
bin_train_df.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label,attack_type
0,172.16.0.5-192.168.50.1-59447-51387-17,172.16.0.5,59447,192.168.50.1,51387,17,2018-12-01 12:27:35.463990,105267,4,0,1398.0,0.0,369.0,330.0,349.5,22.51666,0.0,0.0,0.0,0.0,1.328052e+04,3.799861e+01,35089.000000,60775.064763,105266.0,0.0,105267.0,35089.000000,60775.064763,105266.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,80,0,37.998613,0.0,330.0,369.0,345.6,21.36118,456.3,0,0,0,0,0,0,0,0,0.0,432.0,349.5,0.0,80,0,0,0,0,0,0,4,1398,0,0,-1,-1,3,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SSDP,SSDP
1,172.16.0.5-192.168.50.1-42389-52472-17,172.16.0.5,42389,192.168.50.1,52472,17,2018-12-01 12:27:49.540344,107930,4,0,1398.0,0.0,369.0,330.0,349.5,22.51666,0.0,0.0,0.0,0.0,1.295284e+04,3.706106e+01,35976.666667,62311.682503,107928.0,1.0,107930.0,35976.666667,62311.682503,107928.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-4,0,37.061058,0.0,330.0,369.0,345.6,21.36118,456.3,0,0,0,0,0,0,0,0,0.0,432.0,349.5,0.0,-4,0,0,0,0,0,0,4,1398,0,0,-1,-1,3,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SSDP,SSDP
2,172.16.0.5-192.168.50.1-34492-13376-17,172.16.0.5,34492,192.168.50.1,13376,17,2018-12-01 12:30:45.112955,2,2,0,1000.0,0.0,500.0,500.0,500.0,0.00000,0.0,0.0,0.0,0.0,5.000000e+08,1.000000e+06,2.000000,0.000000,2.0,2.0,2.0,2.000000,0.000000,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,1000000.000000,0.0,500.0,500.0,500.0,0.00000,0.0,0,0,0,0,0,0,0,0,0.0,750.0,500.0,0.0,40,0,0,0,0,0,0,2,1000,0,0,-1,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SSDP,SSDP
3,172.16.0.5-192.168.50.1-53150-28438-17,172.16.0.5,53150,192.168.50.1,28438,17,2018-12-01 12:28:11.476330,47,2,0,802.0,0.0,401.0,401.0,401.0,0.00000,0.0,0.0,0.0,0.0,1.706383e+07,4.255319e+04,47.000000,0.000000,47.0,47.0,47.0,47.000000,0.000000,47.0,47.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,42553.191489,0.0,401.0,401.0,401.0,0.00000,0.0,0,0,0,0,0,0,0,0,0.0,601.5,401.0,0.0,0,0,0,0,0,0,0,2,802,0,0,-1,-1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SSDP,SSDP
4,172.16.0.5-192.168.50.1-44373-37345-17,172.16.0.5,44373,192.168.50.1,37345,17,2018-12-01 12:33:01.077479,0,2,0,766.0,0.0,383.0,383.0,383.0,0.00000,0.0,0.0,0.0,0.0,inf,inf,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2125437950,0,0.000000,0.0,383.0,383.0,383.0,0.00000,0.0,0,0,0,0,0,0,0,0,0.0,574.5,383.0,0.0,-2125437950,0,0,0,0,0,0,2,766,0,0,-1,-1,1,-1062718975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SSDP,SSDP


In [8]:
train_df['Protocol'].value_counts()

17    1822559
6      377265
0         176
Name: Protocol, dtype: int64

In [13]:
train_df['Inbound'][:100000].value_counts()

1    99949
0       51
Name: Inbound, dtype: int64

In [7]:
bin_train_arr = train_df.loc[train_df['Label'] != 'BENIGN'].values
bin_test_arr = test_df.loc[test_df['Label'] != 'BENIGN'].values

In [7]:
del train_df
del test_df

In [9]:
for filename in os.listdir(os.path.join(JAN_PARENT_DIR)):
    if filename.endswith('.csv'):
        print(f'{JAN_PARENT_DIR}/{filename}')

./cicddos_data/CSV-01-12/01-12/DrDoS_SSDP.csv
./cicddos_data/CSV-01-12/01-12/DrDoS_NetBIOS.csv
./cicddos_data/CSV-01-12/01-12/DrDoS_UDP.csv
./cicddos_data/CSV-01-12/01-12/TFTP.csv
./cicddos_data/CSV-01-12/01-12/Syn.csv
./cicddos_data/CSV-01-12/01-12/DrDoS_MSSQL.csv
./cicddos_data/CSV-01-12/01-12/DrDoS_SNMP.csv
./cicddos_data/CSV-01-12/01-12/DrDoS_LDAP.csv
./cicddos_data/CSV-01-12/01-12/DrDoS_NTP.csv
./cicddos_data/CSV-01-12/01-12/UDPLag.csv
./cicddos_data/CSV-01-12/01-12/DrDoS_DNS.csv


In [14]:
def append_to_ndarr(csv_path, filename, bin_train_arr_, bin_test_arr_):
    attack_type = filename.split(".")[0].split("_")[-1]
    print(f'Reading [{filename}]')
    benign_df = pd.read_csv(f"{csv_path}", low_memory=False)
#         benign_df = pd.read_csv(os.path.join(JAN_PARENT_DIR, "DrDoS_NTP.csv"), low_memory=False)
    benign_df = benign_df.loc[benign_df[' Label'] == 'BENIGN']

    print(f"Read Complete [{attack_type}].\nProcessing [{attack_type}].")

    benign_df.drop("Unnamed: 0", axis=1, inplace=True)
    benign_df["attack_type"] = attack_type

    row_count = len(benign_df.index)
    threshold = int(row_count*0.9)

    bin_train_arr_ = np.append(bin_train_arr_, benign_df.iloc[:threshold].values, axis=0)
    bin_test_arr_ = np.append(bin_test_arr_, benign_df.iloc[threshold:].values, axis=0)

    del benign_df

    print(f"Appended {attack_type}.")
    print()
    
    return bin_train_arr_, bin_test_arr_

In [15]:
bin_train_arr, bin_test_arr = append_to_ndarr('./cicddos_data/CSV-01-12/01-12/DrDoS_NetBIOS.csv',
                                              'DrDoS_NetBIOS.csv',
                                              bin_train_arr,
                                              bin_test_arr)

Reading [DrDoS_NetBIOS.csv]
Read Complete [NetBIOS].
Processing [NetBIOS].
Appended NetBIOS.



In [17]:
bin_train_arr, bin_test_arr = append_to_ndarr('./cicddos_data/CSV-01-12/01-12/DrDoS_UDP.csv',
                                              'DrDoS_UDP.csv',
                                              bin_train_arr,
                                              bin_test_arr)

Reading [DrDoS_UDP.csv]
Read Complete [UDP].
Processing [UDP].
Appended UDP.



In [19]:
bin_train_arr, bin_test_arr = append_to_ndarr('./cicddos_data/CSV-01-12/01-12/Syn.csv',
                                              'Syn.csv',
                                              bin_train_arr,
                                              bin_test_arr)

Reading [Syn.csv]
Read Complete [Syn].
Processing [Syn].
Appended Syn.



In [21]:
bin_train_arr, bin_test_arr = append_to_ndarr('./cicddos_data/CSV-01-12/01-12/DrDoS_MSSQL.csv',
                                              'DrDoS_MSSQL.csv',
                                              bin_train_arr,
                                              bin_test_arr)

Reading [DrDoS_MSSQL.csv]
Read Complete [MSSQL].
Processing [MSSQL].
Appended MSSQL.



In [24]:
bin_train_arr, bin_test_arr = append_to_ndarr('./cicddos_data/CSV-01-12/01-12/DrDoS_SNMP.csv',
                                              'DrDoS_SNMP.csv',
                                              bin_train_arr,
                                              bin_test_arr)

Reading [DrDoS_SNMP.csv]
Read Complete [SNMP].
Processing [SNMP].
Appended SNMP.



In [27]:
bin_train_arr, bin_test_arr = append_to_ndarr('./cicddos_data/CSV-01-12/01-12/DrDoS_LDAP.csv',
                                              'DrDoS_LDAP.csv',
                                              bin_train_arr,
                                              bin_test_arr)

Reading [DrDoS_LDAP.csv]
Read Complete [LDAP].
Processing [LDAP].
Appended LDAP.



In [30]:
bin_train_arr, bin_test_arr = append_to_ndarr('./cicddos_data/CSV-01-12/01-12/DrDoS_NTP.csv',
                                              'DrDoS_NTP.csv',
                                              bin_train_arr,
                                              bin_test_arr)

Reading [DrDoS_NTP.csv]
Read Complete [NTP].
Processing [NTP].
Appended NTP.



In [33]:
bin_train_arr, bin_test_arr = append_to_ndarr('./cicddos_data/CSV-01-12/01-12/UDPLag.csv',
                                              'UDPLag.csv',
                                              bin_train_arr,
                                              bin_test_arr)

Reading [UDPLag.csv]
Read Complete [UDPLag].
Processing [UDPLag].
Appended UDPLag.



In [36]:
bin_train_arr, bin_test_arr = append_to_ndarr('./cicddos_data/CSV-01-12/01-12/DrDoS_DNS.csv',
                                              'DrDoS_DNS.csv',
                                              bin_train_arr,
                                              bin_test_arr)

Reading [DrDoS_DNS.csv]
Read Complete [DNS].
Processing [DNS].
Appended DNS.



In [ ]:
bin_train_arr, bin_test_arr = append_to_ndarr('./cicddos_data/CSV-01-12/01-12/TFTP.csv',
                                              'TFTP.csv',
                                              bin_train_arr,
                                              bin_test_arr)

Reading [TFTP.csv]


In [1]:
len(bin_train_arr)

NameError: name 'bin_train_arr' is not defined

In [8]:
with open('./pkls/bin_train.pkl', 'rb') as f:
    bin_train_arr = pkl.load(f)
    bin_train_df = pd.DataFrame(bin_train_arr, columns=out_columns)
    bin_train_df.to_csv(BIN_TRAIN_DATA_PATH, index=False, mode='w')

with open('./pkls/bin_test.pkl', 'rb') as f:
    bin_test_arr = pkl.load(f)
    bin_test_df = pd.DataFrame(bin_test_arr, columns=out_columns)
    bin_test_df.to_csv(BIN_TEST_DATA_PATH, index=False, mode='w')

In [38]:
with open('./pkls/bin_train.pkl', 'wb') as f:
    pkl.dump(bin_train_arr, f)

with open('./pkls/bin_test.pkl', 'wb') as f:
    pkl.dump(bin_test_arr, f)

In [ ]:
bin_train_df = pd.DataFrame(bin_train_arr, columns=out_columns)
bin_train_df.to_csv(BIN_TRAIN_DATA_PATH, index=False, mode='w')

In [ ]:
for filename in os.listdir(JAN_PARENT_DIR):
    if filename.endswith('.csv'):
        attack_type = filename.split(".")[0].split("_")[-1]
        print(f"Reading CSV [{attack_type}].")
        
        benign_df = pd.read_csv(f"{JAN_PARENT_DIR}/{filename}", low_memory=False)
#         benign_df = pd.read_csv(os.path.join(JAN_PARENT_DIR, "DrDoS_NTP.csv"), low_memory=False)
        benign_df = benign_df.loc[benign_df[' Label'] == 'BENIGN']
        
        print(f"Read Complete [{attack_type}].\nProcessing [{attack_type}].")
        
        benign_df.drop("Unnamed: 0", axis=1, inplace=True)
        benign_df["attack_type"] = attack_type
        
        row_count = len(benign_df.index)
        threshold = int(row_count*0.9)
        
        bin_train_arr = np.append(bin_train_arr, benign_df.iloc[:threshold].values, axis=0)
        bin_test_arr = np.append(bin_test_arr, benign_df.iloc[threshold:].values, axis=0)
        
        del benign_df
        
        print(f"Appended {attack_type}.")
        print()
        
#         break

bin_train_df = pd.DataFrame(bin_train_arr, columns=out_columns)
del bin_train_arr
bin_train_df.to_csv(BIN_TRAIN_DATA_PATH, index=False, mode='w')
del bin_train_df


bin_test_df = pd.DataFrame(bin_test_arr, columns=out_columns)
del bin_test_arr
bin_test_df.to_csv(BIN_TEST_DATA_PATH, index=False, mode='w')
del bin_test_df

print()
print("Files Ready.")

Reading CSV [SSDP].
Read Complete [SSDP].
Processing [SSDP].
Appended SSDP.

Reading CSV [NetBIOS].
Read Complete [NetBIOS].
Processing [NetBIOS].
Appended NetBIOS.

Reading CSV [UDP].
Read Complete [UDP].
Processing [UDP].
Appended UDP.

Reading CSV [TFTP].


In [26]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [5]:
csv_path = './cicddos_data/CSV-01-12/01-12/TFTP.csv'
filename = 'TFTP.csv'

attack_type = filename.split(".")[0].split("_")[-1]
print(f'Reading [{filename}]')
benign_df = pd.read_csv(f"{csv_path}", low_memory=False)
#         benign_df = pd.read_csv(os.path.join(JAN_PARENT_DIR, "DrDoS_NTP.csv"), low_memory=False)
benign_df = benign_df.loc[benign_df[' Label'] == 'BENIGN']

print(f"Read Complete [{attack_type}].\nProcessing [{attack_type}].")

benign_df.drop("Unnamed: 0", axis=1, inplace=True)
benign_df["attack_type"] = attack_type

print("reached here")

row_count = len(benign_df.index)
threshold = int(row_count*0.9)

benign_df.iloc[:threshold].to_csv(BIN_TRAIN_DATA_PATH, mode='a', index=False, header=False)
benign_df.iloc[threshold:].to_csv(BIN_TEST_DATA_PATH, mode='a', index=False, header=False)

Reading [TFTP.csv]
Read Complete [TFTP].
Processing [TFTP].
reached here


NameError: name 'benign_df' is not defined

In [ ]:
def create_bin_dataset(parent_dir, out_dir):
    
    
    
    for filename in os.listdir(parent_dir):
        if filename.endswith(".csv") or filename.endswith(".CSV"):
            csv_file_path = os.path.join(parent_dir, filename)
            attack_type = filename.split(".")[0].split("_")[-1]
            print(f"Reading CSV [{attack_type}].")
            
            df = pd.read_csv(csv_file_path, low_memory=False)
#             df = pd.read_csv(os.path.join(parent_dir, "DrDoS_NTP.csv"), low_memory=False)
            print(f"Read Complete [{attack_type}].\nProcessing [{attack_type}].")
            
            df.drop("Unnamed: 0", axis=1, inplace=True)
            
            df["attack_type"] = attack_type
            

            stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
            for train_index, test_index in stratified_splitter.split(df, df[' Label']):
                train_df = df.loc[train_index]
                test_df = df.loc[test_index]
                
                train_df = train_df[:200000]
                test_df = test_df[:2000]
                print(f"Processing Complete [{attack_type}].")
                
                train_df.to_csv(train_csv_path, mode='a', index=False, header=False)
                test_df.to_csv(test_csv_path, mode='a', index=False, header=False)
                
                print(f"Append Complete [{attack_type}].")
        print()
                
    print("All Done.")